Name: Sarang P. Kadakia

Net id: sk11634

 # Problem 1:
 Training a simple chatbot using a seq-to-seq model (50 points)

In [ ]:
!mkdir -p data
!wget -O data/movie-corpus.zip https://zissou.infosci.cornell.edu/convokit/datasets/movie-corpus/movie-corpus.zip
!unzip -o data/movie-corpus.zip -d data

--2025-03-15 23:38:33--  https://zissou.infosci.cornell.edu/convokit/datasets/movie-corpus/movie-corpus.zip
Resolving zissou.infosci.cornell.edu (zissou.infosci.cornell.edu)... 128.253.51.179
Connecting to zissou.infosci.cornell.edu (zissou.infosci.cornell.edu)|128.253.51.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40854701 (39M) [application/zip]
Saving to: ‘data/movie-corpus.zip’

data/movie-corpus.z 100%[===================>]  38.96M  8.69MB/s    in 4.9s    

2025-03-15 23:38:39 (7.92 MB/s) - ‘data/movie-corpus.zip’ saved [40854701/40854701]

Archive:  data/movie-corpus.zip
   creating: data/movie-corpus/
  inflating: data/movie-corpus/utterances.jsonl  
  inflating: data/movie-corpus/conversations.json  
  inflating: data/movie-corpus/corpus.json  
  inflating: data/movie-corpus/speakers.json  
  inflating: data/movie-corpus/index.json  


wandb: It is Weights & Biases (W&B), a developer-oriented toolset designed specifically for machine learning

In [ ]:
pip install wandb --upgrade

In [ ]:
# importing necessary libraries

import torch
from torch.jit import script, trace
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import json
import wandb
import typing
import time
import numpy as np
import pandas as pd

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


# Loading and Preprocessing the data

In [ ]:
corpus_name = "movie-corpus"
corpus = os.path.join("data", corpus_name)

def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "utterances.jsonl"))

b'{"id": "L1045", "conversation_id": "L1044", "text": "They do not!", "speaker": "u0", "meta": {"movie_id": "m0", "parsed": [{"rt": 1, "toks": [{"tok": "They", "tag": "PRP", "dep": "nsubj", "up": 1, "dn": []}, {"tok": "do", "tag": "VBP", "dep": "ROOT", "dn": [0, 2, 3]}, {"tok": "not", "tag": "RB", "dep": "neg", "up": 1, "dn": []}, {"tok": "!", "tag": ".", "dep": "punct", "up": 1, "dn": []}]}]}, "reply-to": "L1044", "timestamp": null, "vectors": []}\n'
b'{"id": "L1044", "conversation_id": "L1044", "text": "They do to!", "speaker": "u2", "meta": {"movie_id": "m0", "parsed": [{"rt": 1, "toks": [{"tok": "They", "tag": "PRP", "dep": "nsubj", "up": 1, "dn": []}, {"tok": "do", "tag": "VBP", "dep": "ROOT", "dn": [0, 2, 3]}, {"tok": "to", "tag": "TO", "dep": "dobj", "up": 1, "dn": []}, {"tok": "!", "tag": ".", "dep": "punct", "up": 1, "dn": []}]}]}, "reply-to": null, "timestamp": null, "vectors": []}\n'
b'{"id": "L985", "conversation_id": "L984", "text": "I hope so.", "speaker": "u0", "meta": {

# Creating formatted DataFile

In [ ]:
# Splits each line of the file to create lines and conversations
def loadLinesAndConversations(fileName):
    lines = {}
    conversations = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            lineJson = json.loads(line)
            # Extract fields for line object
            lineObj = {}
            lineObj["lineID"] = lineJson["id"]
            lineObj["characterID"] = lineJson["speaker"]
            lineObj["text"] = lineJson["text"]
            lines[lineObj['lineID']] = lineObj

            # Extract fields for conversation object
            if lineJson["conversation_id"] not in conversations:
                convObj = {}
                convObj["conversationID"] = lineJson["conversation_id"]
                convObj["movieID"] = lineJson["meta"]["movie_id"]
                convObj["lines"] = [lineObj]
            else:
                convObj = conversations[lineJson["conversation_id"]]
                convObj["lines"].insert(0, lineObj)
            conversations[convObj["conversationID"]] = convObj

    return lines, conversations


# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations.values():
        # Iterate over all the lines of the conversation
        for i in range(len(conversation["lines"]) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation["lines"][i]["text"].strip()
            targetLine = conversation["lines"][i+1]["text"].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

Calling above functions to create new file

In [ ]:
# Define path to new file
datafile = os.path.join(corpus, "formatted_movie_lines.txt")

delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# Initialize lines dict and conversations dict
lines = {}
conversations = {}
# Load lines and conversations
print("\nProcessing corpus into lines and conversations...")
lines, conversations = loadLinesAndConversations(os.path.join(corpus, "utterances.jsonl"))

# Write new csv file
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

# Print a sample of lines
print("\nSample lines from file:")
printLines(datafile)


Processing corpus into lines and conversations...

Writing newly formatted file...

Sample lines from file:
b'They do to!\tThey do not!\n'
b'She okay?\tI hope so.\n'
b"Wow\tLet's go.\n"
b'"I\'m kidding.  You know how sometimes you just become this ""persona""?  And you don\'t know how to quit?"\tNo\n'
b"No\tOkay -- you're gonna need to learn how to lie.\n"
b"I figured you'd get to the good stuff eventually.\tWhat good stuff?\n"
b'What good stuff?\t"The ""real you""."\n'
b'"The ""real you""."\tLike my fear of wearing pastels?\n'
b'do you listen to this crap?\tWhat crap?\n'
b"What crap?\tMe.  This endless ...blonde babble. I'm like, boring myself.\n"


# Loading and trimming the data

In [ ]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

Now we would do the following:
1. Converting the Unicode strings to ASCII using unicodeToAscii.
2. Converting all letters to lowercase and trimming all non-letter characters except for basic punctuation (normalizeString).  
3. Filtering out sentences with length greater than the MAX_LENGTH threshold (filterPairs)

In [ ]:
MAX_LENGTH = 10  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True if both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using the ``filterPair`` condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 221282 sentence pairs
Trimmed to 64313 sentence pairs
Counting words...
Counted words: 18082

pairs:
['they do to !', 'they do not !']
['she okay ?', 'i hope so .']
['wow', 'let s go .']
['what good stuff ?', 'the real you .']
['the real you .', 'like my fear of wearing pastels ?']
['do you listen to this crap ?', 'what crap ?']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['have fun tonight ?', 'tons']


For achieving faster convergence during training we can also trimm rarely used words out of our vocabulary, i.e decrease the feature space which would soften the difficulty of the function that the model must learn to approximate.

1. Trim words used under MIN_COUNT threshold using the voc.trim function.

2. Filter out pairs with trimmed words.

In [ ]:
MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 7833 / 18079 = 0.4333
Trimmed from 64313 pairs to 53131, 0.8261 of total


# Preparing the data for model training

Here, we have to convert the words in our sentence pairs to their corresponding indexes from the vocabulary and feed this to the models

In [ ]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[  11,  103,   33, 3108,   54],
        [  83,   77,   11,    6,   14],
        [ 104,   22,  179,    2,    2],
        [  17, 2153, 4593,    0,    0],
        [ 898, 1323,   14,    0,    0],
        [  22,   22,    2,    0,    0],
        [2232, 2413,    0,    0,    0],
        [  14,   10,    0,    0,    0],
        [   2,    2,    0,    0,    0]])
lengths: tensor([9, 9, 6, 3, 3])
target_variable: tensor([[ 104,   19,  162, 1482,   54],
        [  17,  381,   11,    6,   14],
        [   7,   11,   50,    2,    2],
        [ 898,   62,  415,    0,    0],
        [  22,  582,  179,    0,    0],
        [2232, 1381, 4593,    0,    0],
        [  14,  182,   14,    0,    0],
        [   2,   10,    2,    0,    0],
        [   0,    2,    0,    0,    0]])
mask: tensor([[ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True, False, False],
        [ True,  True,  Tr

# Defining our models

# Seq2Seq Model
The goal of a seq2seq model is to take a variable-length sequence as an input, and return a variable-length sequence as an output using a fixed-sized model.

Here, we are using two separate RNN's together, where one RNN is an encoder which encodes a variable length input sequence to a fixed-length context vector. Second RNN is a decoder, which takes an input word and the context vector, and returns a guess for the next word in the sequence and a hidden state to use in the next iteration.

# Encoder:
Here, the encoder RNN processes input sequences token by token, generating an output and a hidden state at each step. A bidirectional GRU is used to capture both past and future contexts. Word embeddings map input tokens into a feature space, and padding is handled using pack_padded_sequence and pad_packed_sequence. The final output is a sum of bidirectional GRU outputs.

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size parameters are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,dropout=dropout, bidirectional=True)

    def forward(self, input_seq : torch.Tensor, input_lengths : torch.Tensor):
        # shape[0] = num_layers * num_directions
        # Initiating hidden with zeros
        hidden = torch.zeros(self.n_layers * 2, input_seq.shape[1], self.hidden_size).to(input_seq.device)
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

# Decoder
The decoder RNN generates responses token by token, using the encoder’s context vectors and hidden states until it reaches an EOS token. Attention mechanisms help mitigate information loss in long sequences.

# Bahdanau et al (Local Attention)
Here, attention is calculated using the decoder’s current hidden state and the encoder’s outputs. The output attention weights have the same shape as the input sequence, allowing us to multiply them by the encoder outputs, giving us a weighted sum which indicates the parts of encoder output to pay attention to.

# Luong et al (Global Attention)
1. Here, we consider all of the encoder's hidden states as compared considering the encoder’s hidden state from the current time step.
2. We calculate attention weights, or energies, using the hidden state of the decoder from the current time step only.

In [ ]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        self.attn = nn.Linear(self.hidden_size, hidden_size)
        self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        # Initialize attn_energies
        attn_energies = torch.randn(1,1,1)
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

# Actual Luong Attention Decoder


In [ ]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step : torch.Tensor, last_hidden : torch.Tensor, encoder_outputs : torch.Tensor):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

# Define Training Procedure

# Masked Loss
The maskNLLLoss function computes loss by considering only non-padded elements in the target sequence, ensuring accurate training with padded batches.

In [ ]:
def maskNLLLoss(inp, target, mask):
  nTotal = mask.sum()
  crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
  loss = crossEntropy.masked_select(mask).mean()
  loss = loss.to(device)
  return loss, nTotal.item()

# Single Training Iteration
1. The training function processes a single batch using teacher forcing, which helps the decoder learn faster but requires careful tuning to avoid inference instability.

2. Gradient clipping is applied to prevent the exploding gradient problem, ensuring stable training by capping gradients at a predefined threshold.

In [ ]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip,teacher_forcing_ratio, max_length=MAX_LENGTH):

  # Zero Gradients
  encoder_optimizer.zero_grad()
  decoder_optimizer.zero_grad()

  # Set device options
  input_variable = input_variable.to(device)
  target_variable = target_variable.to(device)
  mask = mask.to(device)

  # Lengths for RNN packing should always be on the CPU
  lengths = lengths.to("cpu")

  # Initialize variables
  loss = 0
  print_losses = []
  n_totals = 0

  # Forward pass through encoder

  encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

  # Create initial decoder input (start with SOS tokens for each sentence)
  decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
  decoder_input = decoder_input.to(device)

  # Set initial decoder hidden state to the encoder's final hidden state
  decoder_hidden = encoder_hidden[:decoder.n_layers]

  # Determine if we are using teacher forcing this iteration
  use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

  # Forward batch of sequences through decoder one time step at a time
  if use_teacher_forcing:
      for t in range(max_target_len):
          decoder_output, decoder_hidden = decoder(
              decoder_input, decoder_hidden, encoder_outputs
          )
          # Teacher forcing: next input is current target
          decoder_input = target_variable[t].view(1, -1)
          # Calculate and accumulate loss
          mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
          loss += mask_loss
          print_losses.append(mask_loss.item() * nTotal)
          n_totals += nTotal
  else:
      for t in range(max_target_len):
          decoder_output, decoder_hidden = decoder(
              decoder_input, decoder_hidden, encoder_outputs
          )
          # No teacher forcing: next input is decoder's own current output
          _, topi = decoder_output.topk(1)
          decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
          decoder_input = decoder_input.to(device)
          # Calculate and accumulate loss
          mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
          loss += mask_loss
          print_losses.append(mask_loss.item() * nTotal)
          n_totals += nTotal

  # Record loss
  wandb.log({"loss": loss.item()})

  # Perform backpropagation
  loss.backward()

  # Clip gradients: gradients are modified in place
  _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
  _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

  # Adjust model weights
  encoder_optimizer.step()
  decoder_optimizer.step()

  return sum(print_losses) / n_totals

# Training iterations


In [ ]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name,teacher_forcing_ratio, profile=False, save_model = False):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0

    # Training loop
    print("Training...")

    # Code for profiling
    if profile:
        myprofiler = torch.profiler.profile(
                schedule=torch.profiler.schedule(wait=1, warmup=10, active=10, repeat=1),
                on_trace_ready=torch.profiler.tensorboard_trace_handler(f'./profiler_logs/{wandb.run.name}.log'),
                record_shapes=True,
                with_stack=True)
        # Start the profiler
        myprofiler.start()

    for iteration in range(start_iteration, n_iteration + 1):


        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip, teacher_forcing_ratio)
        print_loss += loss

        if profile:
            myprofiler.step()

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("\rIteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg),  end="", flush=True)
            print_loss = 0

    # End the profiler
    if profile:
        myprofiler.stop()

    if save_model:
        directory = os.path.join(save_dir, model_name, corpus_name, '{}'.format(wandb.run.name))
        if not os.path.exists(directory):
            os.makedirs(directory)
        torch.save({
            'iteration': iteration,
            'en': encoder.state_dict(),
            'de': decoder.state_dict(),
            'en_opt': encoder_optimizer.state_dict(),
            'de_opt': decoder_optimizer.state_dict(),
            'loss': loss,
            'voc_dict': voc.__dict__,
            'embedding': embedding.state_dict()
        }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))


# Define Evaluation

# Greedy Decoding
Greedy decoding selects the word with the highest softmax probability at each time step, ensuring an optimal choice per step but potentially missing global context.

In [ ]:
# class GreedySearchDecoder(nn.Module):
#   def __init__(self, encoder, decoder):
#     super(GreedySearchDecoder, self).__init__()
#     self.encoder = encoder
#     self.decoder = decoder

#   def forward(self, input_seq, input_length, max_length):

#     # Forward input through encoder model
#     encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)

#     decoder_hidden = encoder_hidden[:self.decoder.n_layers]

#     # Initialize decoder input with SOS_token
#     decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token

#     # Initialize tensors to append decoded words to
#     all_tokens = torch.zeros([0], device=device, dtype=torch.long)
#     all_scores = torch.zeros([0], device=device)

#     # Iteratively decode one word token at a time
#     for _ in range(max_length):
#         # Forward pass through decoder
#         decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)

#         # Obtain most likely word token and its softmax score
#         decoder_scores, decoder_input = torch.max(decoder_output, dim=1)

#         all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
#         all_scores = torch.cat((all_scores, decoder_scores), dim=0)

#         # Prepare current token to be next decoder input (add a dimension)
#         decoder_input = torch.unsqueeze(decoder_input, 0)

#     # Return collections of word tokens and scores
#     return all_tokens, all_scores


In [ ]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    def forward(self, input_seq : torch.Tensor, input_length : torch.Tensor, max_length : int):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:self.decoder.n_layers]
        # Initialize decoder input with SOS_token
        batch_size = input_seq.shape[1]
        device = input_seq.device
        SOS_token = 1
        decoder_input = torch.ones(1, batch_size, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

# Evaluating the text

Here, we will define two functions, i.e. evaluate() & evaluateInput()

1. evaluate() processes an input sentence by converting it into word indexes, obtaining a decoded response using GreedySearchDecoder, and returning the generated output.

2. evaluateInput() serves as the chatbot interface, handling user queries in a loop until exit commands are given.

In [ ]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length = MAX_LENGTH):

  # words -> indexes
  indexes_batch = [indexesFromSentence(voc, sentence)]

  # creating length tensors
  lengths = torch.tensors(len(indexes) for indexes in indexes_batch)

  # transposing dimesnions of batch to match model's expectations
  input_batch = torch.LongTensor(indexes_batch).transpose(0,1)

  input_batch = input_batch.to(device)
  lengths = lengths.to("cpu")

  # decode sentence with searcher
  tokens, scores = searcher(input_batch, lengths, max_length)

  # indexes -> words
  decoded_words = [voc.index2word[token.item()] for token in tokens]

  return decoded_words

def evaluateTrainData(searcher, eval_batches, n_iteration, device, max_length=MAX_LENGTH):
    time_diffs = []
    searcher = searcher.to(device)
    for iteration in range(n_iteration):
        training_batch = eval_batches[iteration]
        input_variable, lengths, target_variable, mask, max_target_len = training_batch
        input_variable = input_variable.to(device)
        if iteration > 5:
            torch.cuda.synchronize()
            start = time.monotonic_ns()
        with torch.no_grad():

                output = searcher(input_variable, lengths, max_length)
        if iteration > 5:
            torch.cuda.synchronize()
            end = time.monotonic_ns()
        if iteration > 5:
            time_diffs.append(end-start)
    return np.array(time_diffs) / 1e6

def evaluateInput(encoder, decoder, searcher, voc):
  input_sentence = ""

  while(1):
    try:
      input_sentence = input("> ")
      print("USER:", input_sentence)

      if input_sentence == "q" or input_sentence == "quit":
        break

      # Normalizing sentence
      input_sentence = normalizeString(input_sentence)

      output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)

      # Formatting and print response sentence
      output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
      print('Bot:', ' '.join(output_words))

    except KeyError:
      print("Unknown word!")

In [ ]:
def train_wandb(config=None, profile = False, save_model = False):
    # run = wandb.init(project="HPML_Lab3", entity="sk11634-new-york-university")
    # config = run.config if config is None else config

    run = wandb.init(project="HPML_HW3_Sarang", entity="sk11634-new-york-university", config = config)
    config = wandb.config if config is None else config
    model_name = 'cb_model'
    attn_model = 'dot'
    #``attn_model = 'general'``
    #``attn_model = 'concat'``
    hidden_size = 500
    encoder_n_layers = 2
    decoder_n_layers = 2
    dropout = 0.1
    batch_size = 64

    print('Building encoder and decoder ...')
    # Initialize word embeddings

    embedding = nn.Embedding(voc.num_words, hidden_size)
    # Initialize encoder & decoder models

    encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
    decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)

    # Use appropriate device
    encoder = encoder.to(device)
    decoder = decoder.to(device)
    print('Models built!')

    # Configure training/optimization
    clip = config.clip
    teacher_forcing_ratio = config.tf_ratio
    learning_rate = config.lr
    decoder_learning_ratio = config.decoder_lrn_ratio
    n_iteration = 4000
    print_every = 1
    save_every = 500

    encoder.train()
    decoder.train()

    # Initialize optimizers
    print('Building optimizers ...')
    optimizer_fn = optim.Adam if config.optimizer == "adam" else optim.SGD
    encoder_optimizer = optimizer_fn(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optimizer_fn(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

    # Configure CUDA to call
    for state in encoder_optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()

    for state in decoder_optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()

    # Run training iterations
    print("Starting Training!")
    trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, teacher_forcing_ratio, profile, save_model)

    # Finishing wandb training
    wandb.finish()

    return encoder, decoder

# Run the model

In [ ]:
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


# Running Training

In [ ]:
from dataclasses import dataclass

@dataclass

class Config:
  lr: float
  optimizer: str
  clip: float
  tf_ratio: float
  decoder_lrn_ratio: float

encoder, decoder = train_wandb(Config(lr = 0.0001, optimizer = "adam", clip = 0.0, tf_ratio = 0.0, decoder_lrn_ratio = 1.0), profile = True, save_model = True)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: sk11634 (sk11634-new-york-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Building encoder and decoder ...
Models built!
Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 4000; Percent complete: 100.0%; Average loss: 8.9701

loss,▄▄▄▁▅▂▃▄▄▂▅▄▄▄▃▅▄▃▃▂▃▄▅▄▃▅▃▃▄█▄▄▄▄▅▅▄▅▃▅
loss,89.73458


In [ ]:
encoder.eval()
decoder.eval()
n_iteration = 100
random.seed(1234)
eval_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(64)]) for _ in range(n_iteration)]

In [ ]:
# Initializing search modules
my_search = GreedySearchDecoder(encoder, decoder)
# print(my_search)
my_gpu = evaluateTrainData(my_search, eval_batches, n_iteration, "cuda")
my_gpu.mean()

11.282072127659575

In [ ]:
my_cpu = evaluateTrainData(my_search, eval_batches, n_iteration, "cpu")
my_cpu.mean()

305.21795924468086

In [ ]:
# Set dropout layers to ``eval`` mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Chat
# evaluateInput(encoder, decoder, searcher, voc)

# W&B Random Search Sweep

In [ ]:
sweep_config = {
    'method': 'random',
    'name': 'Lab3-sweep',
    'metric': {
      'goal': 'minimize',
      'name': 'loss'
    },
    'parameters': {
        'lr': {
            'values': [0.0001, 0.00025, 0.0005, 0.001]
        },
        'optimizer': {
            'values': ["adam", "sgd"]
        },
        'clip': {
            'values': [0, 25, 50, 100]
        },
        'tf_ratio': {
            'values': [0, 0.5, 1.0]
        },
        "decoder_lrn_ratio":{
            "values": [1.0, 3.0, 5.0, 10.0]
        }
    },
    # 'max_sweeps': 25
}

# Initialize the sweep
sweep_id = wandb.sweep(sweep = sweep_config, project="HPML_HW3_Sarang")

Create sweep with ID: 5wsg7ac1
Sweep URL: https://wandb.ai/sk11634-new-york-university/HPML_HW3_Sarang/sweeps/5wsg7ac1


In [ ]:
# Run the sweep
wandb.agent(sweep_id, function = train_wandb, count = 25)

wandb: Agent Starting Run: 6se9vujz with config:
wandb: 	clip: 25
wandb: 	decoder_lrn_ratio: 10
wandb: 	lr: 0.001
wandb: 	optimizer: sgd
wandb: 	tf_ratio: 0


Building encoder and decoder ...
Models built!
Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 4000; Percent complete: 100.0%; Average loss: 4.6418

loss,█▅▆▅▆▃▅▅▄▅▃▃▄▂▄▅▄▁▄▅▃▂▃▄▂▅▃▄▄▂▅▄▄▄▁▂▂▂▃▃
loss,36.89048


wandb: Agent Starting Run: y56mq4a1 with config:
wandb: 	clip: 0
wandb: 	decoder_lrn_ratio: 10
wandb: 	lr: 0.001
wandb: 	optimizer: sgd
wandb: 	tf_ratio: 0


Building encoder and decoder ...
Models built!
Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 4000; Percent complete: 100.0%; Average loss: 8.9666

...

Building encoder and decoder ...
Models built!
Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 4000; Percent complete: 100.0%; Average loss: 3.4699

loss,█▆▆▆▅▄▅▅▅▆▆▃▆▆▅▂▅▅▅▃▆▅▂▂▃▃▂▅▂▃▁▁▆▅▆▁▅▂▂▅
loss,27.76451


wandb: Agent Starting Run: abb519rv with config:
wandb: 	clip: 50
wandb: 	decoder_lrn_ratio: 10
wandb: 	lr: 0.0005
wandb: 	optimizer: sgd
wandb: 	tf_ratio: 0.5


Building encoder and decoder ...
Models built!
Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 4000; Percent complete: 100.0%; Average loss: 4.3885

loss,██▅▃▄▃▃▃▂▃▃▃▂▃▃▃▂▂▂▃▂▁▂▃▂▂▂▂▃▂▃▂▁▁▁▃▃▂▁▃
loss,34.35719


In [ ]:
# !wandb agent sk11634-new-york-university/HPML_Lab3/8k7xgbxi

# Best result - hyperparameter

As per the analysis, values of the hyperparameters that give the best result (Minimum loss of the trained model) is below:

- clip=100.0,
- tf_ratio=1.0,
- lr=0.00025,
- optimizer = "adam",
- decoder_lrn_ratio=10



In [ ]:
best_result = Config(
    clip=100.0,
    tf_ratio=1.0,
    lr=0.00025,
    optimizer = "adam",
    decoder_lrn_ratio=10
)
_, _ = train_wandb(best_result, profile=True, save_model=True)

Building encoder and decoder ...
Models built!
Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 4000; Percent complete: 100.0%; Average loss: 2.3609

loss,▇█▇▆▅▃▅▅▅▄▆▄▄▄▄▄▃▃▃▃▃▂▃▄▃▂▂▂▂▂▃▂▂▃▃▂▂▃▂▁
loss,17.54774


# Problem 2

# Utilizing TorchScript

In [ ]:
#Modified GreedySearchDecoder for scripting the module

class GreedySearchDecoderScript(torch.jit.ScriptModule):
    def __init__(self, encoder, decoder, decoder_n_layers):
        super(GreedySearchDecoderScript, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self._device = device
        self._SOS_token = SOS_token
        self._decoder_n_layers = decoder_n_layers

    __constants__ = ['_device', '_SOS_token', '_decoder_n_layers']

    @torch.jit.script_method
    def forward(self, input_seq : torch.Tensor, input_length : torch.Tensor, max_length : int):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:self._decoder_n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=self._device, dtype=torch.long) * self._SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=self._device, dtype=torch.long)
        all_scores = torch.zeros([0], device=self._device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

## Torchscript the GPU based model

In [ ]:
device = torch.device('cuda')
encoder_gpu = encoder.to(device)
decoder_gpu = decoder.to(device)
encoder_gpu.eval()
decoder_gpu.eval()

LuongAttnDecoderRNN(
  (embedding): Embedding(7836, 500)
  (embedding_dropout): Dropout(p=0.1, inplace=False)
  (gru): GRU(500, 500, num_layers=2, dropout=0.1)
  (concat): Linear(in_features=1000, out_features=500, bias=True)
  (out): Linear(in_features=500, out_features=7836, bias=True)
  (attn): Attn(
    (attn): Linear(in_features=500, out_features=500, bias=True)
  )
)

In [ ]:
# Compiling the whole greedy search model to TorchScript model

test_seq = torch.LongTensor(MAX_LENGTH, 1).random_(0, voc.num_words).to(device)
test_seq_length = torch.LongTensor([test_seq.size()[0]]).cpu()

# Tracing the model
traced_encoder = torch.jit.trace(encoder_gpu, (test_seq, test_seq_length))

# Converting decoder model

# Creating and generating artificial inputs
test_encoder_outputs, test_encoder_hidden = traced_encoder(test_seq, test_seq_length)
test_decoder_hidden = test_encoder_hidden[:decoder.n_layers]
test_decoder_input = torch.LongTensor(1, 1).random_(0, voc.num_words).to(device)

# Tracing the model
traced_decoder = torch.jit.trace(decoder_gpu, (test_decoder_input, test_decoder_hidden, test_encoder_outputs))

torchscript_searcher_gpu = torch.jit.script(GreedySearchDecoderScript(traced_encoder, traced_decoder, decoder.n_layers))

/usr/local/lib/python3.11/dist-packages/torch/jit/_trace.py:165: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /pytorch/build/aten/src/ATen/core/TensorBody.h:489.)
  if a.grad is not None:


In [ ]:
# Printing the graph of the converted model
torchscript_searcher_gpu.graph

graph(%self : __torch__.GreedySearchDecoder,
      %input_seq.1 : Tensor,
      %input_length.1 : Tensor,
      %max_length.1 : int):
  %60 : bool = prim::Constant[value=0]()
  %49 : bool = prim::Constant[value=1]() # <ipython-input-18-d0c786c2d337>:20:8
  %25 : int = prim::Constant[value=4]() # <ipython-input-18-d0c786c2d337>:15:71
  %15 : NoneType = prim::Constant()
  %SOS_token.1 : int = prim::Constant[value=1]() # <ipython-input-18-d0c786c2d337>:12:37
  %33 : int = prim::Constant[value=0]() # <ipython-input-18-d0c786c2d337>:17:34
  %encoder : __torch__.EncoderRNN = prim::GetAttr[name="encoder"](%self)
  %7 : (Tensor, Tensor) = prim::CallMethod[name="forward"](%encoder, %input_seq.1, %input_length.1) # <ipython-input-18-d0c786c2d337>:8:42
  %encoder_outputs.1 : Tensor, %encoder_hidden.1 : Tensor = prim::TupleUnpack(%7)
  %decoder.1 : __torch__.LuongAttnDecoderRNN = prim::GetAttr[name="decoder"](%self)
  %n_layers : int = prim::GetAttr[name="n_layers"](%decoder.1)
  %decoder_hidden.1

# Torchscript the CPU based model

In [ ]:
device = torch.device('cpu')
encoder_cpu = encoder.to(device)
decoder_cpu = decoder.to(device)
encoder_cpu.eval()
decoder_cpu.eval()

In [ ]:
# Compile the whole greedy search model to TorchScript model

test_seq = torch.LongTensor(MAX_LENGTH, 1).random_(0, voc.num_words).to(device)
test_seq_length = torch.LongTensor([test_seq.size()[0]]).cpu()

# Trace the model
traced_encoder = torch.jit.trace(encoder_cpu, (test_seq, test_seq_length))

test_encoder_outputs, test_encoder_hidden = traced_encoder(test_seq, test_seq_length)
test_decoder_hidden = test_encoder_hidden[:decoder.n_layers]
test_decoder_input = torch.LongTensor(1, 1).random_(0, voc.num_words).to(device)

# Trace the model
traced_decoder = torch.jit.trace(decoder_cpu, (test_decoder_input, test_decoder_hidden, test_encoder_outputs))

torchscript_searcher_cpu = torch.jit.script(GreedySearchDecoderScript(traced_encoder, traced_decoder, decoder.n_layers))

In [ ]:
torchscript_searcher_cpu.graph

 # Evaluating the Torchscript model

In [ ]:
# Evaluating the Torchscript model with GPU
my_torchscript_gpu = evaluateTrainData(torchscript_searcher_gpu, eval_batches, n_iteration, "cuda")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1412.)
  return forward_call(*args, **kwargs)


In [ ]:
# Evaluating the Torchscript model with CPU
my_torchscript_cpu = evaluateTrainData(torchscript_searcher_cpu, eval_batches, n_iteration, "cuda")

# Comparing the latency

In [ ]:
df = pd.DataFrame({'Latency on CPU (ms)': [0, 0],
                   'Latency on GPU (ms)': [0, 0]},
                  index=['Pytorch', 'Torchscript'])

df.loc['Pytorch', 'Latency on CPU (ms)'] = my_cpu.mean()
df.loc['Pytorch', 'Latency on GPU (ms)'] = my_gpu.mean()

df.loc['Torchscript', 'Latency on CPU (ms)'] = my_torchscript_cpu.mean()
df.loc['Torchscript', 'Latency on GPU (ms)'] = my_torchscript_gpu.mean()

df.loc['SpeedUp'] = df.iloc[0] / df.iloc[1]

df.to_csv('latency.csv', index=True)

df

<ipython-input-37-2997ea17cddc>:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '305.21795924468086' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc['Pytorch', 'Latency on CPU (ms)'] = my_cpu.mean()
<ipython-input-37-2997ea17cddc>:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '11.282072127659575' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc['Pytorch', 'Latency on GPU (ms)'] = my_gpu.mean()


,Latency on CPU (ms),Latency on GPU (ms)
Pytorch,305.217959,11.282072
Torchscript,21.746694,14.643587
SpeedUp,14.035143,0.770445


# Saving the model

In [ ]:
# Saving and serializing the model for use
torchscript_searcher_gpu.save("my_torchscript_searcher_gpu.pt")
torchscript_searcher_cpu.save("my_torchscript_searcher_cpu.pt")

In [ ]:
# # %%shell
# !cp /content/data/save/cb_model/movie-corpus/woven-sweep-25/checkpoint.tar